 # Back Propogation Neural Network

In [1]:
import yaml
import numpy as np

In [2]:
def load_yaml(filename):
    with open(filename, 'r') as f:
        contents = yaml.full_load(f)
    return contents

In [68]:
class BPNN:
    def __init__(self):
        self.weights = None
        self.outputs = None
        self.delta = None
    
    def error(self,predicted, actual):
        error = actual-predicted
        return error @ error
    
    def transfer(self,x):
        return 1/(1+np.exp(-x))
    
    def transfer_derivative(self,x):
         return x * (1 - x)
    
    def set_random_weights(self, neuron_ns,seed=1):
        self.n = len(neuron_ns)-1
        np.random.seed(seed)
        self.weights = [
            np.random.random(((neuron_ns[i+1], neuron_ns[i]+1)))
            for i in range(self.n)
        ]
        self.outputs = [np.zeros((size+1)) for size in neuron_ns]
        self.delta = [np.zeros((size+1)) for size in neuron_ns]
        for x in self.outputs:
            x[0] = 1

    def forward_propagate(self, inputs):
        assert (self.weights is not None),"weights not given"
        self.outputs[0][1:] = inputs
        for i in range(self.n):
            self.outputs[i+1][1:] = self.transfer(
                self.weights[i] @ self.outputs[i]
            )
        return self.outputs[self.n][1:]

    def backward_propagate_error(self, expected):
        for i in range(self.n, 0, -1):
            if i == self.n:
                errors = expected - self.outputs[i][1:]
            else:
                errors = self.weights[i].T[1:] @ self.delta[i+1][1:]
            self.delta[i][1:] = errors * \
                self.transfer_derivative(self.outputs[i][1:])

    def update_weights(self, l_rate):
        for i in range(self.n):
            self.weights[i] += l_rate * \
                self.delta[i+1][1:, np.newaxis] @ self.outputs[i][np.newaxis, :]

    def fit(self, train_x,train_y, l_rate, n_epoch):
        classes = np.unique(train_y)
        expected = (train_y.reshape(-1,1) == classes).astype(np.uint8)
        for epoch in range(n_epoch):
            sum_error = 0
            for i, row in enumerate(train_x):
                outputs = self.forward_propagate(row)
                sum_error += self.error(outputs, expected[i])
                self.backward_propagate_error(expected[i])
                self.update_weights(l_rate)
            print(f'>epoch={epoch}, lrate={l_rate:.3},'
                  f' error={sum_error:.4}')

    def predict(self, inputs):
        outputs = self.forward_propagate(inputs).argmax()
        return outputs

    def validate(self,x_test,y_test):
        m = x_test.shape[0]
        for i in range(m):
            actual = self.predict(x_test[i])
            expected = y_test[i].astype(int)
            print(f"Expected={expected}, Got={actual}")

In [69]:
dataset = np.array(load_yaml("./datasets/bpnn_dataset.yaml"))
print("Dataset:\n",dataset,sep="\n")
split_ratio = 0.5
split = int(split_ratio*dataset.shape[0])
train, test = dataset[:split],dataset[split:]
train_x, train_y = train[:,:-1], train[:,-1]
test_x, test_y = test[:,:-1], test[:,-1]

Dataset:

[[ 2.7810836   2.550537    0.        ]
 [ 1.46548937  2.36212508  0.        ]
 [ 3.39656169  4.40029353  0.        ]
 [ 1.38807019  1.85022032  0.        ]
 [ 3.06407232  3.00530597  0.        ]
 [ 7.62753121  2.75926224  1.        ]
 [ 5.33244125  2.08862677  1.        ]
 [ 6.92259672  1.77106367  1.        ]
 [ 8.67541865 -0.24206865  1.        ]
 [ 7.67375647  3.50856301  1.        ]
 [ 2.7810836   2.550537    0.        ]
 [ 1.46548937  2.36212508  0.        ]
 [ 3.39656169  4.40029353  0.        ]
 [ 1.38807019  1.85022032  0.        ]
 [ 3.06407232  3.00530597  0.        ]
 [ 7.62753121  2.75926224  1.        ]
 [ 5.33244125  2.08862677  1.        ]
 [ 6.92259672  1.77106367  1.        ]
 [ 8.67541865 -0.24206865  1.        ]
 [ 7.67375647  3.50856301  1.        ]]


In [71]:
b = BPNN()
b.set_random_weights([2,2,2],12)
print("\nInitial Weights:\n")
print(*b.weights,sep="\n")
print("\nTraining:\n")
b.fit(train_x,train_y,.5, 20)
print("\nTrained Weights:\n")
print(*b.weights,sep="\n")
print("\nValidation:\n")
b.validate(test_x,test_y)


Initial Weights:

[[0.15416284 0.7400497  0.26331502]
 [0.53373939 0.01457496 0.91874701]]
[[0.90071485 0.03342143 0.95694934]
 [0.13720932 0.28382835 0.60608318]]

Training:

>epoch=0, lrate=0.5, error=6.465
>epoch=1, lrate=0.5, error=5.533
>epoch=2, lrate=0.5, error=4.924
>epoch=3, lrate=0.5, error=4.59
>epoch=4, lrate=0.5, error=4.304
>epoch=5, lrate=0.5, error=3.984
>epoch=6, lrate=0.5, error=3.635
>epoch=7, lrate=0.5, error=3.267
>epoch=8, lrate=0.5, error=2.888
>epoch=9, lrate=0.5, error=2.519
>epoch=10, lrate=0.5, error=2.187
>epoch=11, lrate=0.5, error=1.898
>epoch=12, lrate=0.5, error=1.65
>epoch=13, lrate=0.5, error=1.438
>epoch=14, lrate=0.5, error=1.259
>epoch=15, lrate=0.5, error=1.108
>epoch=16, lrate=0.5, error=0.9817
>epoch=17, lrate=0.5, error=0.8753
>epoch=18, lrate=0.5, error=0.7854
>epoch=19, lrate=0.5, error=0.7092

Trained Weights:

[[-0.31419445  0.86239965 -1.05449705]
 [ 0.82372542 -1.34886773  1.78507267]]
[[-0.1445459  -1.68269707  2.3485556 ]
 [ 0.23456386 